In [95]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

In [163]:
df = pd.read_csv('stats.csv', index_col = 0)

In [164]:
df

,1 Corners,1 Fouls Committed,1 Fouls Wons,1 Goals,1 Offsides,1 Pass Completion,1 Possesion,1 Red Cards,1 Shot Attempts,1 Shots Blocked,...,2 Possesion,2 Red Cards,2 Shot Attempts,2 Shots Blocked,2 Total Passes,2 Total Shots on Target,2 Total xG,2 Under Pressure,2 Yellow Cards,winner
0,2,1,3,0,1,0.800,0.624,0,2,0,...,0.376,0,8,3,175,1,0.876350,0.171,0,2
1,2,0,3,0,3,0.903,0.658,0,7,2,...,0.342,0,9,6,193,1,0.682699,0.104,0,1
2,1,1,6,1,0,0.793,0.468,0,8,4,...,0.532,0,5,2,291,0,0.456537,0.096,2,2
3,1,2,3,1,0,0.826,0.496,0,5,2,...,0.504,0,5,0,304,3,0.162379,0.105,0,1
4,3,1,1,0,0,0.784,0.320,0,6,1,...,0.680,0,7,0,374,3,0.805794,0.059,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
677,6,0,1,0,0,0.837,0.628,0,12,3,...,0.372,0,0,0,175,0,0.000000,0.206,1,2
678,0,4,4,0,1,0.693,0.340,0,0,0,...,0.660,0,13,3,316,6,1.904855,0.127,1,2
679,2,1,2,1,0,0.571,0.354,0,1,0,...,0.646,0,6,2,268,3,0.675959,0.157,0,1
680,1,1,3,2,0,0.878,0.665,0,7,1,...,0.335,0,2,0,199,1,0.182967,0.131,0,1


In [165]:
def hwin(s1, s2):
    if s1 > s2: 
        return 1
    elif s2 > s1:
        return 2
    else:
        return 0

In [166]:
fhw = []
for i in range(len(df.index)):
    hw = hwin(df['1 Goals'][i], df['2 Goals'][i])
    fhw.append(hw)
df['fh winner'] = fhw

In [169]:
features = df.iloc[: , :-2]
target = df.winner

In [172]:
features

,1 Corners,1 Fouls Committed,1 Fouls Wons,1 Goals,1 Offsides,1 Pass Completion,1 Possesion,1 Red Cards,1 Shot Attempts,1 Shots Blocked,...,2 Pass Completion,2 Possesion,2 Red Cards,2 Shot Attempts,2 Shots Blocked,2 Total Passes,2 Total Shots on Target,2 Total xG,2 Under Pressure,2 Yellow Cards
0,2,1,3,0,1,0.800,0.624,0,2,0,...,0.651,0.376,0,8,3,175,1,0.876350,0.171,0
1,2,0,3,0,3,0.903,0.658,0,7,2,...,0.746,0.342,0,9,6,193,1,0.682699,0.104,0
2,1,1,6,1,0,0.793,0.468,0,8,4,...,0.890,0.532,0,5,2,291,0,0.456537,0.096,2
3,1,2,3,1,0,0.826,0.496,0,5,2,...,0.803,0.504,0,5,0,304,3,0.162379,0.105,0
4,3,1,1,0,0,0.784,0.320,0,6,1,...,0.898,0.680,0,7,0,374,3,0.805794,0.059,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
677,6,0,1,0,0,0.837,0.628,0,12,3,...,0.703,0.372,0,0,0,175,0,0.000000,0.206,1
678,0,4,4,0,1,0.693,0.340,0,0,0,...,0.873,0.660,0,13,3,316,6,1.904855,0.127,1
679,2,1,2,1,0,0.571,0.354,0,1,0,...,0.720,0.646,0,6,2,268,3,0.675959,0.157,0
680,1,1,3,2,0,0.878,0.665,0,7,1,...,0.734,0.335,0,2,0,199,1,0.182967,0.131,0


In [173]:
features.shape

(682, 30)

In [174]:
X_train, X_test, y_train, y_test = train_test_split(features, target)

In [175]:
cl_estimators = {
    'k-Nearest Neighbor': KNeighborsClassifier(), 
    'Gaussian Naive Bayes': GaussianNB(),
    'Decision Tree': DecisionTreeClassifier(),
    'Logistic Regression': LogisticRegression(max_iter = 100000)}

In [176]:
def classifiers_models():
    for name, model in cl_estimators.items(): 
        x = model.fit(X=X_train, y = y_train)
        accuracytest = x.score(X_test, y_test)
        accuracytrain = x.score(X_train, y_train)
        print(name + ": \n\t" + "Classification accuracy on the test data:", f"{accuracytest:.2%}")
        print(f"\tClassification accuracy on the training data: {accuracytrain:.2%}\n")

In [177]:
classifiers_models()

k-Nearest Neighbor: 
	Classification accuracy on the test data: 63.74%
	Classification accuracy on the training data: 73.97%

Gaussian Naive Bayes: 
	Classification accuracy on the test data: 65.50%
	Classification accuracy on the training data: 66.73%

Decision Tree: 
	Classification accuracy on the test data: 59.06%
	Classification accuracy on the training data: 100.00%

Logistic Regression: 
	Classification accuracy on the test data: 72.51%
	Classification accuracy on the training data: 74.36%



In [178]:
def cl_model_comp(features, target):
    for name, model in cl_estimators.items():
      kfold = StratifiedKFold(n_splits = 10, random_state = 300, shuffle = True)
      scores = cross_val_score(model, X=features,y=target,cv=kfold)
      print(f'{name}: \n\t mean accuracy={scores.mean():.2%}, standard deviation={scores.std():.2%}\n')

In [179]:
cl_model_comp(features, target)

k-Nearest Neighbor: 
	 mean accuracy=61.71%, standard deviation=4.79%

Gaussian Naive Bayes: 
	 mean accuracy=62.90%, standard deviation=4.60%

Decision Tree: 
	 mean accuracy=61.31%, standard deviation=4.12%

Logistic Regression: 
	 mean accuracy=71.55%, standard deviation=3.20%



In [180]:
def logistic_tuning():
    param_grid = {"solver":["sag", "saga","liblinear","newton-cg", "lbfgs"]}
    grid_search = GridSearchCV(LogisticRegression(max_iter = 100000), param_grid, cv=10)
    grid_search.fit(X=X_train, y=y_train)
    print("Best parameters: ", grid_search.best_params_)
    print("Training set score with best parameter: ", grid_search.score(X_train, y_train))
    print("Test set score with best parameter: ", grid_search.score(X_test, y_test))

In [181]:
logistic_tuning()

Best parameters:  {'solver': 'liblinear'}
Training set score with best parameter:  0.7279843444227005
Test set score with best parameter:  0.7368421052631579


In [182]:
model_cl = LogisticRegression(solver = "newton-cg",max_iter=100000)
model_cl.fit(X=X_train, y = y_train)
predicted_cl = model_cl.predict(X=features)

In [184]:
predicted_cl

array([2, 1, 1, 1, 2, 2, 2, 1, 2, 1, 1, 1, 1, 2, 2, 2, 1, 1, 2, 1, 1, 1,
       1, 2, 2, 0, 2, 0, 2, 1, 1, 2, 1, 2, 2, 1, 2, 1, 1, 1, 0, 0, 2, 1,
       1, 2, 2, 2, 1, 1, 1, 2, 1, 1, 2, 1, 0, 0, 2, 1, 1, 2, 2, 1, 2, 2,
       2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 0,
       2, 1, 0, 2, 2, 1, 1, 1, 2, 1, 2, 1, 1, 2, 1, 2, 1, 1, 2, 1, 1, 1,
       1, 2, 1, 1, 1, 2, 0, 1, 2, 2, 1, 1, 1, 0, 2, 2, 1, 2, 0, 1, 2, 1,
       1, 2, 2, 2, 2, 2, 0, 1, 2, 1, 2, 1, 1, 1, 2, 1, 2, 2, 1, 1, 2, 1,
       2, 0, 1, 1, 0, 1, 1, 2, 1, 1, 2, 2, 1, 0, 1, 1, 2, 1, 1, 1, 2, 1,
       0, 2, 2, 1, 1, 1, 1, 2, 1, 1, 2, 2, 2, 1, 0, 2, 1, 1, 2, 1, 0, 2,
       1, 2, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 2, 2, 1, 1, 0, 1, 2, 0, 2,
       1, 2, 1, 1, 2, 2, 2, 1, 1, 2, 1, 2, 1, 1, 2, 1, 1, 1, 2, 1, 1, 2,
       1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 2, 1, 1, 1, 2, 2, 1,
       1, 2, 2, 1, 2, 2, 1, 1, 2, 1, 2, 2, 1, 2, 1, 2, 1, 1, 0, 1, 2, 1,
       1, 1, 0, 1, 2, 1, 1, 1, 2, 1, 2, 0, 2, 2, 1,

In [188]:
odds = model_cl.predict_proba(X = features)
odds

array([[0.19643552, 0.11469793, 0.68886655],
       [0.23359683, 0.58214711, 0.18425606],
       [0.1624403 , 0.53642981, 0.3011299 ],
       ...,
       [0.32541346, 0.35051956, 0.32406699],
       [0.02606966, 0.97253718, 0.00139316],
       [0.32777205, 0.07380761, 0.59842034]])

In [192]:
oddf = pd.DataFrame(odds, columns = ['% Tie', '% 1 Win', '% 2 Win'])

In [185]:
df['win prediction'] = predicted_cl

In [186]:
df[(df['winner'] != df['win prediction']) & (df['winner'] != df['fh winner'])]

,1 Corners,1 Fouls Committed,1 Fouls Wons,1 Goals,1 Offsides,1 Pass Completion,1 Possesion,1 Red Cards,1 Shot Attempts,1 Shots Blocked,...,2 Shot Attempts,2 Shots Blocked,2 Total Passes,2 Total Shots on Target,2 Total xG,2 Under Pressure,2 Yellow Cards,winner,fh winner,win prediction
2,1,1,6,1,0,0.793,0.468,0,8,4,...,5,2,291,0,0.456537,0.096,2,2,0,1
5,2,2,5,0,0,0.803,0.620,0,2,0,...,5,1,189,0,0.397853,0.074,1,1,2,2
7,8,2,2,0,0,0.838,0.596,0,15,5,...,2,0,205,1,0.083115,0.180,0,2,0,1
12,2,1,7,3,4,0.770,0.506,0,7,3,...,5,1,250,1,0.194865,0.096,0,2,1,1
22,0,0,3,2,1,0.846,0.324,0,4,2,...,8,2,393,2,0.682793,0.310,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
657,2,0,1,1,2,0.828,0.366,0,3,1,...,9,2,322,5,0.773400,0.087,0,0,1,1
662,5,1,0,1,0,0.876,0.786,0,6,1,...,4,0,112,2,0.428301,0.134,1,0,1,1
663,2,1,3,0,0,0.800,0.489,0,4,0,...,6,1,225,0,1.520363,0.236,0,0,2,2
666,0,0,4,0,3,0.800,0.359,0,1,0,...,5,1,357,1,0.526991,0.056,0,1,0,2


In [194]:
df = pd.concat([df, oddf.reindex(df.index)], axis=1)

In [195]:
df

,1 Corners,1 Fouls Committed,1 Fouls Wons,1 Goals,1 Offsides,1 Pass Completion,1 Possesion,1 Red Cards,1 Shot Attempts,1 Shots Blocked,...,2 Total Shots on Target,2 Total xG,2 Under Pressure,2 Yellow Cards,winner,fh winner,win prediction,% Tie,% 1 Win,% 2 Win
0,2,1,3,0,1,0.800,0.624,0,2,0,...,1,0.876350,0.171,0,2,2,2,0.196436,0.114698,0.688867
1,2,0,3,0,3,0.903,0.658,0,7,2,...,1,0.682699,0.104,0,1,0,1,0.233597,0.582147,0.184256
2,1,1,6,1,0,0.793,0.468,0,8,4,...,0,0.456537,0.096,2,2,0,1,0.162440,0.536430,0.301130
3,1,2,3,1,0,0.826,0.496,0,5,2,...,3,0.162379,0.105,0,1,1,1,0.177071,0.756689,0.066240
4,3,1,1,0,0,0.784,0.320,0,6,1,...,3,0.805794,0.059,0,2,2,2,0.108255,0.025883,0.865862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
677,6,0,1,0,0,0.837,0.628,0,12,3,...,0,0.000000,0.206,1,2,0,1,0.240272,0.587130,0.172598
678,0,4,4,0,1,0.693,0.340,0,0,0,...,6,1.904855,0.127,1,2,2,2,0.057283,0.007572,0.935145
679,2,1,2,1,0,0.571,0.354,0,1,0,...,3,0.675959,0.157,0,1,1,1,0.325413,0.350520,0.324067
680,1,1,3,2,0,0.878,0.665,0,7,1,...,1,0.182967,0.131,0,1,1,1,0.026070,0.972537,0.001393


In [196]:
df.iloc[0]

1 Corners                    2.000000
1 Fouls Committed            1.000000
1 Fouls Wons                 3.000000
1 Goals                      0.000000
1 Offsides                   1.000000
1 Pass Completion            0.800000
1 Possesion                  0.624000
1 Red Cards                  0.000000
1 Shot Attempts              2.000000
1 Shots Blocked              0.000000
1 Total Passes             290.000000
1 Total Shots on Target      0.000000
1 Total xG                   0.069639
1 Under Pressure             0.279000
1 Yellow Cards               0.000000
2 Corners                    6.000000
2 Fouls Committed            2.000000
2 Fouls Wons                 0.000000
2 Goals                      1.000000
2 Offsides                   1.000000
2 Pass Completion            0.651000
2 Possesion                  0.376000
2 Red Cards                  0.000000
2 Shot Attempts              8.000000
2 Shots Blocked              3.000000
2 Total Passes             175.000000
2 Total Shot